In [1]:
import numpy as np
import os
import pandas as pd
from scipy.optimize import minimize, Bounds

We load our example grid and extract some useful data.

In [2]:
input_path = os.path.join("..", "..", "example", "input", "generation", "case118_l2rpn_wcci")
df = pd.read_csv(os.path.join(input_path, "prods_charac.csv"))


n = df.shape[0]
avg_pmaxs = df.groupby(["type"])["Pmax"].mean()
types = avg_pmaxs.index.to_numpy()
avg_pmaxs = avg_pmaxs.to_numpy()
capacity_factor = np.array([30, 95, 15, np.nan, 25])
average_load = 2800
target_energy_mix = np.array([9., 36., 17., 2., 36.])
target_pmax = 10000

In [3]:
def get_nb_pp_by_type(t):
  return df[df["type"] == t].shape[0]

In [4]:
nb_type = types.shape[0]

x = np.ones(2*nb_type+1) * 1000

for i in range(nb_type):
  x[nb_type+1+i] = get_nb_pp_by_type(types[i])

In [5]:
def get_pmaxs(x):
  nb_pp = np.round(x)
  pmaxs = np.zeros(types.shape)
  for i in range(types.shape[0]):
    pmaxs[i] = avg_pmaxs[i] * nb_pp[i]
  return pmaxs

def get_apriori_energy_mix(pmaxs, capacity_factor, average_load):
  apriori_energy_mix = capacity_factor * pmaxs / average_load
  total = np.nansum(apriori_energy_mix)
  if total > 100:
    #print("WARNING: apriori energy mix w/o thermal exceeds average load!")
    apriori_energy_mix[0] -= (total - 100)
    apriori_energy_mix[3] = 0
  else:
    apriori_energy_mix[3] = 100 - total
  return apriori_energy_mix

def split_slack_vars(x):
  n = types.shape[0] + 1
  return x[:n], x[n:]

In [6]:
def objective(x):
  slacks, _ = split_slack_vars(x)
  return slacks.sum()

# x >= 0
def constraint_1(x):
  return x

# x = n
def constraint_2(x):
  _, x = split_slack_vars(x)
  x = np.round(x)
  return x.sum() - n

def constraint_3(x):
  _, x = split_slack_vars(x)
  x = np.round(x)
  return -x.sum() + n

def constraint_4(x):
  slacks, x = split_slack_vars(x)
  slacks = slacks[:-1]
  pmaxs = get_pmaxs(x)
  apriori_energy_mix = get_apriori_energy_mix(pmaxs, capacity_factor, average_load)
  return apriori_energy_mix - target_energy_mix + slacks

def constraint_5(x):
  slacks, x = split_slack_vars(x)
  slacks = slacks[-1]
  pmaxs = get_pmaxs(x)
  return -pmaxs.sum() + target_pmax + slacks

constraints = [
  {"type": "ineq", "fun": constraint_1},
  {"type": "ineq", "fun": constraint_2},
  {"type": "ineq", "fun": constraint_3},
  {"type": "ineq", "fun": constraint_4},
  {"type": "ineq", "fun": constraint_5}
]

In [7]:
bounds = Bounds(0, np.inf)
options = {"maxiter": 500_000}
res = minimize(objective, x, constraints=constraints, method="cobyla", options=options)
res

     fun: 50.144285714285715
   maxcv: 2.6416480330535448e-17
 message: 'Optimization terminated successfully.'
    nfev: 2717
  status: 1
 success: True
       x: array([-1.02590908e-22,  8.85714286e+00,  1.30014286e+01, -1.62116547e-19,
        2.82857143e+01, -2.64164803e-17,  5.36277586e+00,  2.21084534e+00,
        1.56187666e+01,  2.08004864e+01,  1.76750613e+01])

In [8]:
x = res.x

In [9]:
constraint_1(x)

array([-1.02590908e-22,  8.85714286e+00,  1.30014286e+01, -1.62116547e-19,
        2.82857143e+01, -2.64164803e-17,  5.36277586e+00,  2.21084534e+00,
        1.56187666e+01,  2.08004864e+01,  1.76750613e+01])

In [10]:
constraint_2(x)

0.0

In [11]:
constraint_3(x)

0.0

In [12]:
constraint_4(x)

array([ 4.39285714,  0.        ,  0.        , 45.75142857,  0.        ])

In [13]:
_, x = split_slack_vars(x)

In [14]:
def get_nb_power_plants(x):
  nb_pp = np.round(x)
  power_plants = {
      "hydro": 0,
      "nuclear": 0,
      "solar": 0,
      "thermal": 0,
      "wind": 0,
      "total": int(nb_pp.sum())
    }
  for i in range(nb_pp.shape[0]):
    key = list(power_plants.keys())[int(i)]
    power_plants[key] += int(nb_pp[i])
  return power_plants

In [15]:
get_nb_power_plants(x)

{'hydro': 5, 'nuclear': 2, 'solar': 16, 'thermal': 21, 'wind': 18, 'total': 62}

In [16]:
pmaxs = get_pmaxs(x)
em = get_apriori_energy_mix(pmaxs, capacity_factor, average_load)
error = np.abs(target_energy_mix - em).sum()
print(f"Target energy mix: {target_energy_mix}")
print(f"Actual energy mix: {em}")
print(f"Difference between target and actual energy mix: {error:.2f}%")

Target energy mix: [ 9. 36. 17.  2. 36.]
Actual energy mix: [13.39285714 27.14285714  3.99857143 47.75142857  7.71428571]
Difference between target and actual energy mix: 100.29%


In [17]:
pmaxs

array([1250.        ,  800.        ,  746.4       , 2959.09090909,
        864.        ])

In [18]:
pmaxs.sum()

6619.490909090909